In [25]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mayankkathane/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
986,"The whole point of making this film, one of th...",positive
188,Whatever rating I give BOOM is only because of...,negative
155,I'm working my way through the Horror Classics...,negative
975,"In this unlikely love triangle, set in 19th ce...",positive
674,This film is a fun little private eye detectiv...,positive


In [27]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [28]:
df = normalize_text(df)
df.head()

Error during text normalization: 
**********************************************************************
  Resource wordnet not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('wordnet')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load corpora/wordnet

  Searched in:
    - '/Users/mayankkathane/nltk_data'
    - '/Users/mayankkathane/Projects/End-to-End-MLOps-Sentiment-Analysis-Pipeline-with-Automated-CI-CD/sentiment-analysis-mlops/nltk_data'
    - '/Users/mayankkathane/Projects/End-to-End-MLOps-Sentiment-Analysis-Pipeline-with-Automated-CI-CD/sentiment-analysis-mlops/share/nltk_data'
    - '/Users/mayankkathane/Projects/End-to-End-MLOps-Sentiment-Analysis-Pipeline-with-Automated-CI-CD/sentiment-analysis-mlops/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
*****************************************************

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/Users/mayankkathane/nltk_data'
    - '/Users/mayankkathane/Projects/End-to-End-MLOps-Sentiment-Analysis-Pipeline-with-Automated-CI-CD/sentiment-analysis-mlops/nltk_data'
    - '/Users/mayankkathane/Projects/End-to-End-MLOps-Sentiment-Analysis-Pipeline-with-Automated-CI-CD/sentiment-analysis-mlops/share/nltk_data'
    - '/Users/mayankkathane/Projects/End-to-End-MLOps-Sentiment-Analysis-Pipeline-with-Automated-CI-CD/sentiment-analysis-mlops/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************

In [ ]:
df['sentiment'].value_counts()

sentiment
negative    259
positive    241
Name: count, dtype: int64

In [ ]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [ ]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
993,trying picture pitch dark angel i m thinking m...,0
155,working way horror classics movie pack collect...,0
32,became cult movie chinese college students tho...,0
558,waaaaay much frustrating watch waiting whole d...,0
651,saying movie extremely hard follow frustrating...,0


In [ ]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [31]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/imayank45/End-to-End-MLOps-Sentiment-Analysis-Pipeline-with-Automated-CI-CD.mlflow')
dagshub.init(repo_owner='imayank45', repo_name='End-to-End-MLOps-Sentiment-Analysis-Pipeline-with-Automated-CI-CD', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-11-24 19:18:54,459 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/imayank45/End-to-End-MLOps-Sentiment-Analysis-Pipeline-with-Automated-CI-CD "HTTP/1.1 200 OK"


Initialized MLflow to track repo "imayank45/End-to-End-MLOps-Sentiment-Analysis-Pipeline-with-Automated-CI-CD"

2025-11-24 19:18:54,464 - INFO - Initialized MLflow to track repo "imayank45/End-to-End-MLOps-Sentiment-Analysis-Pipeline-with-Automated-CI-CD"


Repository imayank45/End-to-End-MLOps-Sentiment-Analysis-Pipeline-with-Automated-CI-CD initialized!

2025-11-24 19:18:54,466 - INFO - Repository imayank45/End-to-End-MLOps-Sentiment-Analysis-Pipeline-with-Automated-CI-CD initialized!


<Experiment: artifact_location='mlflow-artifacts:/16a5547f06824fc8b15e11e042ca0f90', creation_time=1763991346062, experiment_id='0', last_update_time=1763991346062, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [33]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        joblib.dump(model, "model.pkl")
        mlflow.log_artifact("model.pkl")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-11-24 19:21:44,120 - INFO - Starting MLflow run...
2025-11-24 19:21:45,070 - INFO - Logging preprocessing parameters...
2025-11-24 19:21:46,295 - INFO - Initializing Logistic Regression model...
2025-11-24 19:21:46,295 - INFO - Fitting the model...
2025-11-24 19:21:46,305 - INFO - Model training complete.
2025-11-24 19:21:46,305 - INFO - Logging model parameters...
2025-11-24 19:21:46,805 - INFO - Making predictions...
2025-11-24 19:21:46,807 - INFO - Calculating evaluation metrics...
2025-11-24 19:21:46,814 - INFO - Logging evaluation metrics...
2025-11-24 19:21:48,669 - INFO - Saving and logging the model...
2025-11-24 19:21:49,782 - INFO - Model training and logging completed in 4.71 seconds.
2025-11-24 19:21:49,783 - INFO - Accuracy: 0.76
2025-11-24 19:21:49,783 - INFO - Precision: 0.7586206896551724
2025-11-24 19:21:49,784 - INFO - Recall: 0.7333333333333333
2025-11-24 19:21:49,784 - INFO - F1 Score: 0.7457627118644068


🏃 View run masked-swan-259 at: https://dagshub.com/imayank45/End-to-End-MLOps-Sentiment-Analysis-Pipeline-with-Automated-CI-CD.mlflow/#/experiments/0/runs/25727e7c016e4986b1b39a97da949bed
🧪 View experiment at: https://dagshub.com/imayank45/End-to-End-MLOps-Sentiment-Analysis-Pipeline-with-Automated-CI-CD.mlflow/#/experiments/0
